In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models

from google.colab import drive

drive.mount('/content/drive')


In [ ]:
train_pixel_size_path = '/content/drive/My Drive/B3_code/ultrasound_dataset/training_set_pixel_size_and_HC.csv'
test_pixel_size_path = '/content/drive/My Drive/B3_code/ultrasound_dataset/test_set_pixel_size.csv'

train_df = pd.read_csv(train_pixel_size_path)
test_df = pd.read_csv(test_pixel_size_path)

train_df.head()


In [ ]:
images_dir = '/content/drive/My Drive/B3_code/ultrasound_dataset/training_set'
annotations_dir = '/content/drive/My Drive/B3_code/ultrasound_dataset/training_set'


In [ ]:
def load_image_and_annotation(image_path, annotation_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0

    annotation = cv2.imread(annotation_path, cv2.IMREAD_GRAYSCALE)
    annotation = cv2.resize(annotation, (224, 224))

    return image, annotation


In [ ]:
def prepare_data(df, images_dir, annotations_dir):
    images = []
    labels = []

    for _, row in df.iterrows():
        image_path = os.path.join(images_dir, row['filename'])
        annotation_path = os.path.join(
            annotations_dir,
            row['filename'].replace('.png', '_Annotation.png')
        )

        image, _ = load_image_and_annotation(image_path, annotation_path)
        images.append(image)
        labels.append(row['head circumference (mm)'])

    return np.array(images), np.array(labels)

X_train, y_train = prepare_data(train_df, images_dir, annotations_dir)
X_train = X_train.reshape(-1, 224, 224, 1)


In [ ]:
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = build_model()
model.summary()


In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - loss: 9864.9033 - mae: 82.1748 - val_loss: 28099.7520 - val_mae: 160.3346
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - loss: 1960.4337 - mae: 35.9413 - val_loss: 21252.3574 - val_mae: 137.9269
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - loss: 1535.7736 - mae: 32.2755 - val_loss: 19881.3145 - val_mae: 133.8416
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - loss: 1088.2583 - mae: 26.5608 - val_loss: 16223.2070 - val_mae: 120.2343
Epoch 5/10


In [ ]:
def prepare_test_data(df, images_dir):
    images = []

    for _, row in df.iterrows():
        image_path = os.path.join(images_dir, row['filename'])
        image, _ = load_image_and_annotation(image_path, image_path)
        images.append(image)

    return np.array(images)

X_test = prepare_test_data(
    test_df,
    '/content/drive/My Drive/B3_code/ultrasound_dataset/test_set'
)
X_test = X_test.reshape(-1, 224, 224, 1)


In [ ]:
y_pred = model.predict(X_test)

test_df['predicted_head_circumference'] = y_pred
test_df.to_csv(
    '/content/drive/My Drive/B3_code/ultrasound_dataset/test_predictions.csv',
    index=False
)

test_df[['filename', 'predicted_head_circumference']].head()
